## Lendo arquivodo HDFS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

hive_context = HiveContext(sc)

spark = SparkSession \
    .builder \
    .appName("Teste API") \
    .enableHiveSupport() \
    .getOrCreate()

geo = spark.read.orc('/datalake/dadosbrutos/geolocation_correios.orc')

## Consultando coordenadas por cidade e estados distintos 

In [ ]:
!pip install geopy

In [3]:
from geopy.geocoders import Nominatim
import time

start_time = time.time()
# Cidade e estado distintos
cidades_ufs = geo.select('cep_5_digitos','cidade','uf').distinct().collect()
qtde_cidades_ufs = geo.select('cep_5_digitos','cidade','uf').distinct().count()

coords = []
counter = 0

# Consulta com o Geopy qual as coordenadas para cada conjunto cidade, estado.
for linha in cidades_ufs:
    print(f"{counter}º Consulta de {qtde_cidades_ufs}", end="\r")
    try:
        geolocator = Nominatim(user_agent="test_app", timeout=15)
        location = geolocator.geocode(f'{linha["cidade"]}, {linha["uf"]}')
        coords.append([linha['cep_5_digitos'], location.raw["lat"], location.raw["lon"]])
        counter += 1
    except Exception as e:
        print(e)
        pass

# Cria o dataframe com as coordenadas de cada cep
coords_df = spark.createDataFrame(coords, schema=["cep_5_digitos","lat", "lon"])
coords_df.show(truncate=False)
duration = time.time() - start_time
print(f"Tempo total: {duration/60} minutes")

'NoneType' object has no attribute 'raw'
'NoneType' object has no attribute 'raw'
'NoneType' object has no attribute 'raw'
'NoneType' object has no attribute 'raw'
+-----+-------------------+-------------------+
|lat  |lon                |_3                 |
+-----+-------------------+-------------------+
|72805|-16.2535456        |-47.9500276        |
|26315|-22.7156978        |-43.555754         |
|95480|-29.0489517        |-50.145412         |
|77019|-10.1837852        |-48.3336423        |
|96070|-31.7699736        |-52.3410161        |
|85887|-25.2404183        |-53.9835004        |
|16901|-20.896505         |-51.3742765        |
|79837|-22.2206145        |-54.812208         |
|17063|-22.3218102        |-49.0705863        |
|72901|-15.9465415        |-48.2616527        |
|14820|-21.7297099        |-48.1052092        |
|09770|-23.7080345        |-46.5506747        |
|88930|-28.894109999999998|-49.725390543121556|
|65260|-2.000269          |-44.528114         |
|29200|-20.6718755  

## Grava o resultado no HDFS

In [8]:
coords_df.write.orc(f'/datalake/dadosbrutos/geolocation_coords.orc', 'overwrite')

## JOIN dos endereços obtidos no site Correios com as coordenadas

In [23]:
geo_final = geo.select('cep_5_digitos','cidade', 'uf')\
   .dropDuplicates()\
   .join(
        coords_df,
        on=["cep_5_digitos"],
        how="inner"
    )

In [24]:
geo_final.show()

+-------------+--------------------+---+-------------------+-------------------+
|cep_5_digitos|              cidade| uf|                lat|                lon|
+-------------+--------------------+---+-------------------+-------------------+
|        72805|            Luziânia| GO|        -16.2535456|        -47.9500276|
|        26315|           Queimados| RJ|        -22.7156978|         -43.555754|
|        95480|      Cambará do Sul| RS|        -29.0489517|         -50.145412|
|        77019|              Palmas| TO|        -10.1837852|        -48.3336423|
|        96070|             Pelotas| RS|        -31.7699736|        -52.3410161|
|        85887|          Matelândia| PR|        -25.2404183|        -53.9835004|
|        16901|           Andradina| SP|         -20.896505|        -51.3742765|
|        79837|            Dourados| MS|        -22.2206145|         -54.812208|
|        17063|               Bauru| SP|        -22.3218102|        -49.0705863|
|        72901|Santo Antônio

## Grava resultado no HDFS

In [28]:
geo_final.write.orc(f'/datalake/dadosbrutos/geolocation_correios_coords.orc', 'overwrite')